In [ ]:
from drfp import DrfpEncoder
from src.clip import EnzymeReactionCLIP

rxn_smiles = [
    "CO.O[C@@H]1CCNC1.[C-]#[N+]CC(=O)OC>>[C-]#[N+]CC(=O)N1CC[C@@H](O)C1",
    "CCOC(=O)C(CC)c1cccnc1.Cl.O>>CCC(C(=O)O)c1cccnc1",
]

fps = DrfpEncoder.encode(rxn_smiles)

In [2]:
fps

[array([0, 0, 0, ..., 0, 0, 0], shape=(2048,), dtype=uint8),
 array([0, 0, 0, ..., 0, 0, 0], shape=(2048,), dtype=uint8)]

In [11]:
am_rxn = "[C:1][C:2]>>[C:1]=[C:2]"
rxn = "CC>>C=C"

am_fp = DrfpEncoder.encode(am_rxn)
fp = DrfpEncoder.encode(rxn)
(am_fp[0] == fp[0]).all()

np.False_

In [ ]:
ckpt = 
model = EnzymeReactionCLIP.load_from_checkpoint(
            checkpoint_path=ckpt
        )